In [34]:
import boto3
from botocore.config import Config
import sys
import os
import time
import requests
from bs4 import BeautifulSoup


In [35]:
ohio_region  = Config(region_name="us-east-2")
north_virginia_region  = Config(region_name="us-east-1")

In [36]:
ohio_ec2 = boto3.resource('ec2', config=ohio_region)
north_virginia_ec2 = boto3.resource('ec2', config=north_virginia_region)

In [37]:
script_mysql = '''#!/bin/bash
sudo apt update
sudo apt install mysql-server -y
sudo mysql -uroot -e "ALTER USER 'root'@'localhost' IDENTIFIED WITH mysql_native_password BY 'cloud';CREATE USER 'root'@'%' IDENTIFIED BY 'cloud';GRANT ALL PRIVILEGES ON *.* TO 'root'@'%';FLUSH PRIVILEGES;CREATE DATABASE tasks;"
sudo sed -i 's/bind-address/#banana/g' /etc/mysql/mysql.conf.d/mysqld.cnf
sudo sed -i 's/mysqlx/#banana/g' /etc/mysql/mysql.conf.d/mysqld.cnf
sudo ufw allow 3306/tcp
sudo systemctl restart mysql
'''

In [38]:
security_group = ohio_ec2.create_security_group(
    Description='string',
    GroupName='SecurityGroup2',
    TagSpecifications=[
        {
            'ResourceType': 'security-group',
            'Tags': [
                {
                    'Key': 'Name',
                    'Value': 'mysql'
                },
            ]
        },
    ],
)




security_group.authorize_ingress(IpProtocol="tcp",CidrIp="0.0.0.0/0",FromPort=0,ToPort=10000)

security_group2 = north_virginia_ec2.create_security_group(
    Description='string',
    GroupName='SecurityGroup2',
    TagSpecifications=[
        {
            'ResourceType': 'security-group',
            'Tags': [
                {
                    'Key': 'Name',
                    'Value': 'mysql'
                },
            ]
        },
    ],
)




security_group2.authorize_ingress(IpProtocol="tcp",CidrIp="0.0.0.0/0",FromPort=0,ToPort=10000)

{'Return': True,
 'SecurityGroupRules': [{'SecurityGroupRuleId': 'sgr-04254e42f37910b48',
   'GroupId': 'sg-04820a29c081369a3',
   'GroupOwnerId': '903616414837',
   'IsEgress': False,
   'IpProtocol': 'tcp',
   'FromPort': 0,
   'ToPort': 10000,
   'CidrIpv4': '0.0.0.0/0'}],
 'ResponseMetadata': {'RequestId': '1706cd7f-0856-4319-bcb5-8ec08942dc43',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '1706cd7f-0856-4319-bcb5-8ec08942dc43',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '721',
   'date': 'Wed, 24 Nov 2021 13:15:35 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [39]:
instance_ohio = ohio_ec2.create_instances(
        ImageId="ami-00399ec92321828f5",
        MinCount=1,
        MaxCount=1,
        InstanceType="t2.micro",
        KeyName="jpgianfaldoni",
        SecurityGroupIds = [security_group.group_id],
        UserData = script_mysql
    )



In [40]:
instance_ohio[0].wait_until_running()
instance_ohio[0].reload()
public_ip_ohio = instance_ohio[0].public_ip_address
time.sleep(10)

In [41]:
script_django_mysql = f"""#!/bin/bash
cd /home/ubuntu
sudo apt update
sudo apt-get install python3-dev default-libmysqlclient-dev build-essential -y
git clone https://github.com/raulikeda/tasks.git
cd tasks
sudo sed -i 's/node1/{public_ip_ohio}/g' portfolio/settings.py
sudo sed -i 's/django.db.backends.postgresql/django.db.backends.mysql/g' portfolio/settings.py
sudo sed -i 's/5432/3306/g' portfolio/settings.py
sudo sed -i "s/'USER': 'cloud'/'USER': 'root'/g" portfolio/settings.py
echo "mysqlclient" >> requirements.txt
./install.sh
sudo reboot 
"""

In [42]:
north_virginia = north_virginia_ec2.create_instances(
        ImageId="ami-09e67e426f25ce0d7",
        MinCount=1,
        MaxCount=1,
        InstanceType="t2.micro",
        KeyName="jpgianfaldoni",
        SecurityGroupIds = [security_group2.group_id],
        UserData = script_django_mysql
    )

In [43]:
north_virginia[0].wait_until_running()
north_virginia[0].reload()


In [44]:
nv_ip = north_virginia[0].public_ip_address

In [45]:
nv_ip

'52.90.87.11'

In [46]:
nv_response = "Fail"
client_nv = requests.session()
try_counter = 0

In [47]:
while nv_response == "Fail":
    time.sleep(1)
    try:
        nv_response = client_nv.get(f"http://{nv_ip}:8080/tasks/")
    except:
        nv_response = "Fail"
    if try_counter > 300:
        break
    try_counter += 1
    print(try_counter)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27


In [48]:
django_img = north_virginia[0].create_image(Name='django_img3')


In [49]:
while django_img.state != 'available':
    django_img.reload()
north_virginia[0].terminate()

{'TerminatingInstances': [{'CurrentState': {'Code': 32,
    'Name': 'shutting-down'},
   'InstanceId': 'i-09a52451d6a1bf536',
   'PreviousState': {'Code': 16, 'Name': 'running'}}],
 'ResponseMetadata': {'RequestId': '2637d704-27a3-4578-9fca-cc411994f0d2',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '2637d704-27a3-4578-9fca-cc411994f0d2',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'vary': 'accept-encoding',
   'content-type': 'text/xml;charset=UTF-8',
   'transfer-encoding': 'chunked',
   'date': 'Wed, 24 Nov 2021 13:23:10 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [50]:
autoscaling  = boto3.client('autoscaling', config=north_virginia_region)
elb = boto3.client('elbv2', config=north_virginia_region)


In [51]:
create_launch_response = autoscaling.create_launch_configuration(
    LaunchConfigurationName='LCV999',
    ImageId= django_img.id,
    KeyName='jpgianfaldoni',
    SecurityGroups=[
        security_group2.id,
    ],
    BlockDeviceMappings=[
                        {
                            'DeviceName': '/dev/sda1',
                            'Ebs': {
                                'DeleteOnTermination': True,
                                'VolumeSize': 8,
                                'VolumeType': 'gp2'
                            },
                        },
                ],
    InstanceType='t2.micro',
    
)


In [52]:
create_autoscaling_response = autoscaling.create_auto_scaling_group(
    AutoScalingGroupName='AutoScalingV1',
    LaunchConfigurationName='LCV999',
    MinSize=1,
    MaxSize=3,
    DesiredCapacity=3,
    DefaultCooldown=60,
    AvailabilityZones=[
        'us-east-1a',
        'us-east-1b',
        'us-east-1c',
        'us-east-1d',
        'us-east-1e',
        'us-east-1f',
    ]
)

In [53]:
create_target_response = elb.create_target_group(
    Name='Tgroup5',
    Protocol='HTTP',
    Port=8080,
    VpcId='vpc-ebd11591',
    HealthCheckPath = '/tasks/',
    HealthCheckProtocol='HTTP',
    HealthCheckEnabled=True,
    HealthCheckIntervalSeconds=30,
    HealthCheckTimeoutSeconds=15,
    HealthyThresholdCount=5,
    UnhealthyThresholdCount=5,
    TargetType='instance',
    IpAddressType='ipv4'
)

In [54]:
tg_arn = create_target_response['TargetGroups'][0]['TargetGroupArn']

In [55]:
create_lb_response = elb.create_load_balancer(
    Name='Elb3',
    Subnets=[
        'subnet-54296d1e',
        'subnet-7ebe0d50',
        'subnet-13fa4b4f',
        'subnet-e0f44387',
    ],

    SecurityGroups=[
        security_group2.id,
    ],
    Type='application',
    IpAddressType='ipv4',
)

In [56]:
lb_arn = create_lb_response['LoadBalancers'][0]['LoadBalancerArn']

In [57]:
ab_response = autoscaling.attach_load_balancer_target_groups(
    AutoScalingGroupName='AutoScalingV1',
    TargetGroupARNs=[
        tg_arn,
    ]
)

In [58]:
listener_response = elb.create_listener(
    LoadBalancerArn= lb_arn,
    DefaultActions=[
        {
            'Type': 'forward',
            'TargetGroupArn': tg_arn,
        },
    ],
    Port = 80,
    Protocol = 'HTTP',
)

In [59]:
lb_url = create_lb_response['LoadBalancers'][0]['DNSName']
lb_url

'Elb3-1845620628.us-east-1.elb.amazonaws.com'

In [60]:
URL = f'http://{lb_url}/admin/login/?next=/admin/'

In [61]:
URL

'http://Elb3-1845620628.us-east-1.elb.amazonaws.com/admin/login/?next=/admin/'

In [78]:
client = requests.session()

client.get(URL) 

csrftoken = client.cookies['csrftoken']

    
login_data = dict(username='cloud', password='cloud', csrfmiddlewaretoken=csrftoken, next='/admin/')
r = client.post(URL, data=login_data, headers=dict(Referer=URL))
cookies = dict(sessionid=client.cookies.get('sessionid'))
print(cookies)

{'sessionid': 'nud7mvmd5vgzjab1h1hpndm7d18rwwvm'}


In [79]:
URL_add_user = f"http://{lb_url}/admin/auth/user/add/"
URL_add_group = f"http://{lb_url}/admin/auth/group/add/"

In [80]:
def add_user(URL, username, password1, password2,cookies):
    client = requests.session()
    client.get(URL)  # sets cookie
    csrftoken = client.cookies['csrftoken']
    post_data = dict(csrfmiddlewaretoken=csrftoken, username=username, password1=password1,
                     password2 =password2, _save = "Save")
    r2 = client.post(URL, data=post_data, headers=dict(Referer=URL), cookies = cookies)
    return r2
    

In [81]:
def delete_user(user_id, cookies):
    URL_delete = f"http://{lb_url}/admin/auth/user/{user_id}/delete/"
    client = requests.session()
    client.get(URL_delete)  # sets cookie
    csrftoken = client.cookies['csrftoken']
    post_data = dict(csrfmiddlewaretoken=csrftoken,post = "yes")
    r2 = client.post(URL_delete, data=post_data, headers=dict(Referer=URL_delete), cookies = cookies)
    return r2

In [82]:
def add_group(URL, groupname ,cookies):
    client = requests.session()
    client.get(URL)  # sets cookie
    csrftoken = client.cookies['csrftoken']
    post_data = dict(csrfmiddlewaretoken=csrftoken, name=groupname, _save = "Save")
    r2 = client.post(URL, data=post_data, headers=dict(Referer=URL), cookies = cookies)
    return r2

In [83]:
def delete_group(group_id, cookies):
    URL_delete = f"http://{lb_url}/admin/auth/group/{group_id}/delete/"
    client = requests.session()
    client.get(URL_delete)  # sets cookie
    csrftoken = client.cookies['csrftoken']
    post_data = dict(csrfmiddlewaretoken=csrftoken,post = "yes")
    r2 = client.post(URL_delete, data=post_data, headers=dict(Referer=URL_delete), cookies = cookies)
    return r2

In [84]:
add_user(URL_add_user, 'Joao4', 'Banana888', 'Banana888', cookies)

<Response [200]>

In [ ]:
delete_user(4, cookies)

In [85]:
add_group(URL_add_group, 'GroupJoao', cookies)

<Response [200]>

In [ ]:
delete_group(1, cookies)

In [87]:
### DELETE ALL
##OHIO
instance_ohio[0].terminate()
instance_ohio[0].wait_until_terminated()
security_group.delete()

{'ResponseMetadata': {'RequestId': '09b0fd93-2ec8-41eb-a9b7-4267a9648b40',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '09b0fd93-2ec8-41eb-a9b7-4267a9648b40',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '239',
   'date': 'Wed, 24 Nov 2021 13:32:36 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [88]:
##NORTH VIRGINIA
autoscaling.delete_auto_scaling_group(AutoScalingGroupName ='AutoScalingV1', ForceDelete = True)

{'ResponseMetadata': {'RequestId': 'e111abf9-6d33-4e99-b00e-e79415b084d1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'e111abf9-6d33-4e99-b00e-e79415b084d1',
   'content-type': 'text/xml',
   'content-length': '231',
   'date': 'Wed, 24 Nov 2021 13:33:30 GMT'},
  'RetryAttempts': 0}}

In [89]:
django_img.deregister()

{'ResponseMetadata': {'RequestId': '601a87d2-fb5f-4acd-bf6f-f54fa3703622',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '601a87d2-fb5f-4acd-bf6f-f54fa3703622',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '231',
   'date': 'Wed, 24 Nov 2021 13:33:32 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [90]:
elb.delete_load_balancer(LoadBalancerArn= lb_arn)

{'ResponseMetadata': {'RequestId': '4d37a244-ab34-4a6e-b086-8421109d91bd',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '4d37a244-ab34-4a6e-b086-8421109d91bd',
   'content-type': 'text/xml',
   'content-length': '262',
   'date': 'Wed, 24 Nov 2021 13:33:35 GMT'},
  'RetryAttempts': 0}}

In [92]:
elb.delete_target_group(TargetGroupArn= tg_arn)

{'ResponseMetadata': {'RequestId': '8f296a7b-e362-4372-95ee-d57eecce97b8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '8f296a7b-e362-4372-95ee-d57eecce97b8',
   'content-type': 'text/xml',
   'content-length': '259',
   'date': 'Wed, 24 Nov 2021 13:33:48 GMT'},
  'RetryAttempts': 0}}

In [93]:
autoscaling.delete_launch_configuration(LaunchConfigurationName ='LCV999')

{'ResponseMetadata': {'RequestId': '38fd13bd-ce61-4ed9-8113-a7fe31eeabc1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '38fd13bd-ce61-4ed9-8113-a7fe31eeabc1',
   'content-type': 'text/xml',
   'content-length': '237',
   'date': 'Wed, 24 Nov 2021 13:33:49 GMT'},
  'RetryAttempts': 0}}

In [95]:
security_group2.delete()

ClientError: An error occurred (DependencyViolation) when calling the DeleteSecurityGroup operation: resource sg-04820a29c081369a3 has a dependent object